In [1]:
import tensorflow as tf
import numpy as np
import librosa
import os
import pyaudio
import wave
import scipy.io.wavfile as wav
import numpy as np


In [13]:

def snore_detector():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(128, 44, 1)),
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Preprocess the audio file into a spectrogram
def preprocess_audio(file_path):
    y, sr = librosa.load(file_path)
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
    spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
    spectrogram = np.expand_dims(spectrogram, axis=-1)
         
    return spectrogram

# Load the data
def load_data(data_path):
    s = 0
    n = 0
    x_train = []
    y_train = []
    failed_files = []  # To store the file names that failed spectrogram extraction
    for folder_name in os.listdir(data_path):
        folder_path = os.path.join(data_path, folder_name)
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            try:
                spectrogram = preprocess_audio(file_path)
                x_train.append(spectrogram)
                if folder_name == 'snoring_nd':
                    y_train.append(1)
                    s += 1
                elif folder_name == 'non':
                    y_train.append(0)
                    n += 1
            except:
                failed_files.append(file_path)
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    print("Successful spectrogram extractions:", len(x_train))
    print("Failed spectrogram extractions:", len(failed_files))
    print("Snoring samples:", s)
    print("Non-snoring samples:", n)
    return x_train, y_train

# Split the data into training and testing sets
def split_data(x, y, test_size=0.3):
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=42)
    return x_train, y_train, x_test, y_test
        


# Train the model
def train_model(model, x_train, y_train, x_test, y_test, epochs=10):
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs)
    model.save('snoring_local3.h5')
    return model

# Evaluate the model
def evaluate_model(model, x_test, y_test):
    loss, accuracy = model.evaluate(x_test, y_test)
    return loss, accuracy
    
# Run the program
if __name__ == '__main__':
    data_path = "snoringds"
    x, y = load_data(data_path)
    x=x[:5558]
    x_train, y_train, x_test, y_test = split_data(x, y)
    model = snore_detector()
    model = train_model(model, x_train, y_train, x_test, y_test)
    loss, accuracy = evaluate_model(model, x_test, y_test)
    print('Test loss:', loss)
    print('Test accuracy:', accuracy)


Successful spectrogram extractions: 5631
Failed spectrogram extractions: 0
Snoring samples: 5082
Non-snoring samples: 476
Epoch 1/10
122/122 [==============================] - 18s 135ms/step - loss: 0.4199 - accuracy: 0.9167 - val_loss: 0.1914 - val_accuracy: 0.9161
Epoch 2/10
122/122 [==============================] - 16s 133ms/step - loss: 0.1379 - accuracy: 0.9411 - val_loss: 0.1246 - val_accuracy: 0.9460
Epoch 3/10
122/122 [==============================] - 16s 133ms/step - loss: 0.0796 - accuracy: 0.9686 - val_loss: 0.0677 - val_accuracy: 0.9748
Epoch 4/10
122/122 [==============================] - 16s 134ms/step - loss: 0.0514 - accuracy: 0.9799 - val_loss: 0.0848 - val_accuracy: 0.9664
Epoch 5/10
122/122 [==============================] - 16s 135ms/step - loss: 0.0423 - accuracy: 0.9853 - val_loss: 0.0717 - val_accuracy: 0.9670
Epoch 6/10
122/122 [==============================] - 17s 137ms/step - loss: 0.0287 - accuracy: 0.9889 - val_loss: 0.0700 - val_accuracy: 0.9706
Epoch 7/

In [ ]:
data_path = "snoringds"
x, y = load_data(data_path)

In [221]:
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 1024
RECORD_SECONDS = 1
WAVE_OUTPUT_FILENAME = "output.wav"

audio = pyaudio.PyAudio()

stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,
                frames_per_buffer=CHUNK)

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

stream.stop_stream()
stream.close()
audio.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(audio.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

# Read the wave file
sample_rate, data = wav.read('output.wav')

# Calculate the current number of frames
num_frames = len(data)

# Calculate the number of frames to add
frames_to_add = 44100 - num_frames

# Create a zero-filled array with the same number of channels
padding = np.zeros((frames_to_add, 44032))
z=np.zeros(padding.shape[0])
# Concatenate the padding with the original data
padded_data = np.concatenate((data,z), axis=0)

# Write the padded data to a new wave file
wav.write('my_padded_wave_file.wav', sample_rate, padded_data)
model = tf.keras.models.load_model('snoring_local.h5')
def preprocess_audio(file_path):
    y, sr = librosa.load(file_path)
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
    spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
    spectrogram = np.expand_dims(spectrogram, axis=-1)
    return spectrogram

model = tf.keras.models.load_model('snoring_local.h5')

new_audio_file_path = 'my_padded_wave_file.wav'
new_spectrogram = preprocess_audio(new_audio_file_path)

# Make a prediction
prediction = model.predict(np.array([new_spectrogram]))
if prediction[0][0] > 0.01:
    print('The audio file contains snoring.',prediction)
else:
    print('The audio file does not contain snoring.',prediction)


1/1 [==============================] - 0s 66ms/step
The audio file does not contain snoring. [[0.00072291]]


In [186]:
model = tf.keras.models.load_model('snoring_local.h5')
def preprocess_audio(file_path):
    y, sr = librosa.load(file_path)
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
    spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
    spectrogram = np.expand_dims(spectrogram, axis=-1)
    return spectrogram

model = tf.keras.models.load_model('snoring_local.h5')

new_audio_file_path = 'my_padded_wave_file.wav'
new_spectrogram = preprocess_audio(new_audio_file_path)

# Make a prediction
prediction = model.predict(np.array([new_spectrogram]))
if prediction[0][0] > 0.01:
    print('The audio file contains snoring.',prediction)
else:
    print('The audio file does not contain snoring.',prediction)


1/1 [==============================] - 0s 63ms/step
The audio file does not contain snoring. [[0.00185027]]


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


def evaluate_model(model, x_test, y_test):
    y_pred = model.predict(x_test)
    y_pred = np.round(y_pred).flatten()
    confusion = confusion_matrix(y_test, y_pred)
    return confusion

# Run the program
if __name__ == '__main__':
    data_path = "snoringds"
    x, y = load_data(data_path)
    x_train, y_train, x_test, y_test = split_data(x, y)
    model = snore_detector()
    model = train_model(model, x_train, y_train, x_test, y_test)
    confusion = evaluate_model(model, x_test, y_test)
    print('Confusion Matrix:')
    print(confusion)
    
    # Plotting the confusion matrix
    labels = ['Non-snoring', 'Snoring']
    sns.heatmap(confusion, annot=True, cmap='Blues', fmt='d', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()


In [3]:
!pip install PyQt5.QtWebEngineWidgets

ERROR: Could not find a version that satisfies the requirement PyQt5.QtWebEngineWidgets (from versions: none)
ERROR: No matching distribution found for PyQt5.QtWebEngineWidgets


In [7]:
!pip install ui_main

ERROR: Could not find a version that satisfies the requirement ui_main (from versions: none)
ERROR: No matching distribution found for ui_main


In [ ]:
import numpy as np
import os
import librosa
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

def snore_detector():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(128, 44, 1)),
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Preprocess the audio file into a spectrogram
def preprocess_audio(file_path):
    y, sr = librosa.load(file_path)
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
    spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
    spectrogram = np.expand_dims(spectrogram, axis=-1)
         
    return spectrogram

# Load the data
def load_data(data_path):
    s = 0
    n = 0
    x_train = []
    y_train = []
    failed_files = []  # To store the file names that failed spectrogram extraction
    for folder_name in os.listdir(data_path):
        folder_path = os.path.join(data_path, folder_name)
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            try:
                spectrogram = preprocess_audio(file_path)
                x_train.append(spectrogram)
                if folder_name == 'snoring_nd':
                    y_train.append(1)
                    s += 1
                elif folder_name == 'non':
                    y_train.append(0)
                    n += 1
            except:
                failed_files.append(file_path)
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    print("Successful spectrogram extractions:", len(x_train))
    print("Failed spectrogram extractions:", len(failed_files))
    print("Snoring samples:", s)
    print("Non-snoring samples:", n)
    return x_train, y_train

# Split the data into training and testing sets
def split_data(x, y, test_size=0.3):
    x_train, y_train, x_test, y_test = split_data(x, y, test_size=0.25)
    return x_train, y_train, x_test, y_test

# Train the model
def train_model(model, x_train, y_train, x_test, y_test, epochs=10):
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs)
    model.save('snoring_local3.h5')
    return model

# Evaluate the model
def evaluate_model(model, x_test, y_test):
    loss, accuracy = model.evaluate(x_test, y_test)
    return loss, accuracy

# Run the program
if __name__ == '__main__':
    data_path = "snoringds"
    x, y = load_data(data_path)
    x = x[:5743]
    x_train, y_train, x_test, y_test = split_data(x, y)
    model = snore_detector()
    model = train_model(model, x_train, y_train, x_test, y_test)
    loss, accuracy = evaluate_model(model, x_test, y_test)
    print('Test loss:', loss)
    print('Test accuracy:', accuracy)

    # Calculate predictions for the test data
    y_pred = model.predict(x_test)
    y_pred = (y_pred > 0.5).astype(int)

    # Compute the confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Visualize the confusion matrix
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()


C:\Users\Adithya\anaconda3\envs\snorring\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Adithya\anaconda3\envs\snorring\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\Adithya\anaconda3\envs\snorring\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [16]:
x_train = []
y_train = []
n=0
failed_files = []  # To store the file names that failed spectrogram extraction
for folder_name in os.listdir("snoringds"):
    folder_path = os.path.join("snoringds", folder_name)
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if folder_name == 'non':
            y_train.append(0)



In [17]:
y_train

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [4]:
import numpy as np
import os
import librosa
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

def snore_detector():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(128, 44, 1)),
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Preprocess the audio file into a spectrogram
def preprocess_audio(file_path):
    y, sr = librosa.load(file_path)
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
    spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
    spectrogram = np.expand_dims(spectrogram, axis=-1)
         
    return spectrogram

# Load the data
def load_data(data_path):
    s = 0
    n = 0
    x_train = []
    y_train = []
    failed_files = []  # To store the file names that failed spectrogram extraction
    for folder_name in os.listdir(data_path):
        folder_path = os.path.join(data_path, folder_name)
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            try:
                spectrogram = preprocess_audio(file_path)
                x_train.append(spectrogram)
                if folder_name == 'snoring_nd':
                    y_train.append(1)
                    s += 1
                elif folder_name == 'non':
                    y_train.append(0)
                    n += 1
            except:
                failed_files.append(file_path)
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    print("Successful spectrogram extractions:", len(x_train))
    print("Failed spectrogram extractions:", len(failed_files))
    print("Snoring samples:", s)
    print("Non-snoring samples:", n)
    return x_train, y_train

# Split the data into training and testing sets
def split_data(x, y, test_size=0.3):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=42)
    return x_train, y_train, x_test, y_test

# Train the model
def train_model(model, x_train, y_train, x_test, y_test, epochs=10):
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs)
    model.save('snoring_local3.h5')
    return model

# Evaluate the model
def evaluate_model(model, x_test, y_test):
    loss, accuracy = model.evaluate(x_test, y_test)
    return loss, accuracy

# Run the program
if __name__ == '__main__':
    data_path = "snoringds"
    x, y = load_data(data_path)
    x = x[:5550]
    x_train, y_train, x_test, y_test = split_data(x, y)
    model = snore_detector()
    model = train_model(model, x_train, y_train, x_test, y_test)
    loss, accuracy = evaluate_model(model, x_test, y_test)
    print('Test loss:', loss)
    print('Test accuracy:', accuracy)

    # Calculate predictions for the test data
    y_pred = model.predict(x_test)
    y_pred = (y_pred > 0.5).astype(int)

    # Compute the confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Visualize the confusion matrix
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()


Successful spectrogram extractions: 5743
Failed spectrogram extractions: 0
Snoring samples: 5082
Non-snoring samples: 588


ValueError: Found input variables with inconsistent numbers of samples: [5550, 5670]

In [2]:
!pip install seaborn